In [9]:
import pandas as pd
import selfies as sf
from rdkit import Chem
from rdkit.Chem import rdmolops

class molecules:
    def __init__(self, csv_path, nrows=None):
        """
        initialize the processor with path to CSV file
        CSV should contain at least a 'smiles' and 'cartesian_coordinates' columns
        """
        self.data = pd.read_csv(csv_path, nrows=nrows)
        self.validate_input()
    
    def validate_input(self):
        """validate that the input data contains required columns"""
        if 'smiles' not in self.data.columns or 'cartesian_coordinates' not in self.data.columns:
            raise ValueError("CSV must contain a 'smiles' and a 'cartesian_coordinates' columns")
    
    def preprocess_molecules(self):
        """convert SMILES to RDKit molecules and compute needed properties"""
        # Convert SMILES to RDKit molecules
        try: self.data['molecules'] = self.data['smiles'].apply(Chem.MolFromSmiles)
        except ValueError: print("invalid SMILES string")

        # add hydrogens to the molecules
        self.data['molecules_with_h'] = self.data['molecules'].apply(lambda mol: Chem.AddHs(mol))
        
        # compute the adjacency matrices
        self.data['adjacency_matrices'] = self.data['molecules'].apply(lambda mol: rdmolops.GetAdjacencyMatrix(mol))

        # count hydrogens
        self.data['hydrogen_count'] = self.data['molecules_with_h'].apply(lambda mol: mol.GetNumAtoms()) - self.data['molecules'].apply(lambda mol: mol.GetNumAtoms())

        # Convert SMILES to SELFIES
        self.data['selfies'] = self.data['smiles'].apply(lambda x: sf.encoder(x))
        
    def export_training_data(self, output_path):
        """export processed data to CSV"""
        # create export dataframe without RDKit mol objects
        export_df = self.data['selfies', 'hydrogen_count', 'coordinates']
        export_df.to_csv(output_path, index=False)

# Example usage
if __name__ == "__main__":
    # initialize processor
    molecules = molecules('data/split_1.csv', 10)
    print(molecules.data.iloc[0])

    """
    # export the data
    #processor.training_data('processed_molecules.csv')
    print(molecules.data.iloc[0]['atomic_numbers'])
    print(molecules.data.iloc[0]['adjacency_matrices'])
    print(molecules.data.iloc[0]['hydrogen_count'])
    print(molecules.data.iloc[0]['selfies'])
    """
    
    # print some statistics
    print("number of processed molecules:", len(molecules.data))

cid                                                                 193373
cartesian_coordinates    [1.0854899213475149, -0.20413998519187976, 0.2...
atomic_numbers           [6, 6, 6, 6, 6, 6, 6, 6, 8, 8, 6, 6, 8, 8, 1, ...
smiles                                            OC(=O)CC(CCCC(C(=O)O)C)C
Name: 0, dtype: object
number of processed molecules: 10


In [41]:
import numpy as np

mol = Chem.MolFromSmiles(sf.decoder('[O][=O]'))
mol_h = Chem.AddHs(mol)

adj_matrix = rdmolops.GetAdjacencyMatrix(mol)

print(f"\nA:")
print(adj_matrix)

print(f"\nA²:")
print(adj_matrix @ adj_matrix)

print(f"\nA² - diagonalūs elementai:")
print(adj_matrix @ adj_matrix * (1 - np.eye(len(adj_matrix[0])).astype(int)))

print(f"\nA³:")
print(adj_matrix @ adj_matrix @ adj_matrix)

print(f"\nA³ - A elementai:")
print(adj_matrix @ adj_matrix @ adj_matrix * (1 - adj_matrix))


A:
[[0 1]
 [1 0]]

A²:
[[1 0]
 [0 1]]

A² - diagonalūs elementai:
[[0 0]
 [0 0]]

A³:
[[0 1]
 [1 0]]

A³ - A elementai:
[[0 0]
 [0 0]]


In [32]:
print(sf.get_semantic_constraints())

{'H': 1, 'F': 1, 'Cl': 1, 'Br': 1, 'I': 1, 'B': 3, 'B+1': 2, 'B-1': 4, 'O': 2, 'O+1': 3, 'O-1': 1, 'N': 3, 'N+1': 4, 'N-1': 2, 'C': 4, 'C+1': 5, 'C-1': 3, 'P': 5, 'P+1': 6, 'P-1': 4, 'S': 6, 'S+1': 7, 'S-1': 5, '?': 8}


In [34]:
print(sf.get_semantic_robust_alphabet())

{'[=Ring2]', '[H]', '[S-1]', '[S]', '[#B-1]', '[C-1]', '[#C]', '[=N]', '[P-1]', '[B+1]', '[=B]', '[=S+1]', '[#S+1]', '[O-1]', '[=S]', '[F]', '[N+1]', '[#N+1]', '[B-1]', '[Branch1]', '[#P-1]', '[=Ring3]', '[I]', '[N]', '[=B-1]', '[=N-1]', '[#B]', '[P+1]', '[=Branch3]', '[Cl]', '[C+1]', '[#P+1]', '[B]', '[=S-1]', '[N-1]', '[Branch3]', '[#Branch3]', '[=O]', '[Br]', '[=N+1]', '[=C-1]', '[#C-1]', '[=O+1]', '[#N]', '[=C]', '[=P+1]', '[O]', '[Ring1]', '[=C+1]', '[#O+1]', '[O+1]', '[=P-1]', '[=Branch1]', '[#Branch1]', '[Ring2]', '[Ring3]', '[=P]', '[#S]', '[S+1]', '[#Branch2]', '[#S-1]', '[=Branch2]', '[=Ring1]', '[=B+1]', '[Branch2]', '[C]', '[#P]', '[P]', '[#C+1]'}


In [49]:
def calculate_missing_hydrogens(selfies_string):
    """
    Calculate the number of missing hydrogens for each atom in a SELFIES string.
    
    Args:
        selfies_string (str): A valid SELFIES string
        
    Returns:
        list: List of tuples containing (atom_index, atom_symbol, missing_hydrogens)
    """
    valences = sf.get_semantic_constraints()
    
    tokens = sf.split_selfies(selfies_string)
            
    # Process tokens to calculate missing hydrogens
    results = []
    atom_index = 0
    b = 0
    bc = 0
    
    for token in tokens:
        if not token.startswith('[') or not token.endswith(']'):
            continue
            
        if b == 3:
            
        # remove brackets
        token = token[1:-1]
        
        # skip branch tokens
        if 'Branch1' in token:
            b = 1
            continue
        elif 'Branch2' in token:
            b = 2
            continue
        elif 'Branch3' in token:
            b = 3
            continue
            
        # Parse atom and charge
        base_atom = ''
        charge = 0
        bonds = 0
        
        # Handle double and triple bonds
        if token.startswith('='):
            bonds = 2
            token = token[1:]
        elif token.startswith('#'):
            bonds = 3
            token = token[1:]
            
        # Extract charge if present
        if '+' in token:
            parts = token.split('+')
            base_atom = parts[0]
            charge = int(parts[1]) if len(parts[1]) > 0 else 1
        elif '-' in token:
            parts = token.split('-')
            base_atom = parts[0]
            charge = -int(parts[1]) if len(parts[1]) > 0 else -1
        else:
            base_atom = token
            
        # Calculate missing hydrogens if it's a valid atom
        if base_atom in valences:
            # Adjust valence based on charge
            adjusted_valence = valences[base_atom]
            if base_atom == 'N' and charge == 1:
                adjusted_valence = 4
            
            # Calculate missing hydrogens
            missing_h = adjusted_valence - bonds
            
            results.append((atom_index, base_atom, missing_h))
            atom_index += 1
            
    return results

# Example usage and test
def test_calculate_hydrogens():
    selfies = "[O-1][C][=Branch1][C][=O][C][C][Branch1][#Branch2][C][N+1][Branch1][C][C][Branch1][C][C][C][O][C][=Branch1][C][=O][C]"
    results = calculate_missing_hydrogens(selfies)
    
    print("Atom Index | Atom | Missing Hydrogens")
    print("-" * 35)
    for idx, atom, hydrogens in results:
        print(f"{idx:^10}|{atom:^6}|{hydrogens:^17}")
        
    return results

In [50]:
test_calculate_hydrogens()

Atom Index | Atom | Missing Hydrogens
-----------------------------------
    0     |  O   |        2        
    1     |  C   |        4        
    2     |  C   |        4        
    3     |  O   |        0        
    4     |  C   |        4        
    5     |  C   |        4        
    6     |  C   |        4        
    7     |  N   |        4        
    8     |  C   |        4        
    9     |  C   |        4        
    10    |  C   |        4        
    11    |  C   |        4        
    12    |  C   |        4        
    13    |  O   |        2        
    14    |  C   |        4        
    15    |  C   |        4        
    16    |  O   |        0        
    17    |  C   |        4        


[(0, 'O', 2),
 (1, 'C', 4),
 (2, 'C', 4),
 (3, 'O', 0),
 (4, 'C', 4),
 (5, 'C', 4),
 (6, 'C', 4),
 (7, 'N', 4),
 (8, 'C', 4),
 (9, 'C', 4),
 (10, 'C', 4),
 (11, 'C', 4),
 (12, 'C', 4),
 (13, 'O', 2),
 (14, 'C', 4),
 (15, 'C', 4),
 (16, 'O', 0),
 (17, 'C', 4)]

In [60]:
selfies_string = sf.encoder("c1ccccc1")  # Cyclopropane (3-membered ring)
print("SELFIES Encoding:", selfies_string)

# Decode and inspect the symbols
decoded_symbols = sf.split_selfies(selfies_string)
print("Decoded SELFIES Symbols:", decoded_symbols)

SELFIES Encoding: [C][=C][C][=C][C][=C][Ring1][=Branch1]
Decoded SELFIES Symbols: <generator object split_selfies at 0x00000151C4D448B0>
